In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd


#loading the dataset

data = gutenberg.raw('shakespeare-hamlet.txt')

with open('hamlet.txt','w') as file:
      file.write(data)

[nltk_data] Downloading package gutenberg to /usr/share/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [6]:
##Data Preprocessing

import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

#load_dataset
with open('hamlet.txt','r') as file:
    text = file.read().lower()
    
token = Tokenizer()
token.fit_on_texts([text])  ##  A Tokenizer
tot_word = len(token.word_index)+1  ##no of total words with tokenizer and converted to one bsed index
tot_word

4818

creating input sequences

In [7]:
inp_seq=[]
for line in text.split('\n'):
    tok_list = token.texts_to_sequences([line])[0]
    for i in range(1,len(tok_list)):
        n_gram_seq = tok_list[:i+1]
        inp_seq.append(n_gram_seq)

In [8]:
inp_seq

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [9]:
##finding the maxing length in the sequences to apply minimal padding 

max_seq_len = max([len(x) for x in inp_seq])

inp_seq = pad_sequences(inp_seq,maxlen=max_seq_len,padding='pre')

In [10]:
x, y = inp_seq[:, :-1], inp_seq[:, -1]
##giving inp and output

In [11]:
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]], dtype=int32)

In [12]:
y = keras.utils.to_categorical(y,num_classes=tot_word)

converting y into a single vector instead of a number

In [13]:
y ##highly sparse because of single output word

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=24)

# Training the Model

In [17]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout

##Creating the model

model = Sequential()
model.add(Embedding(tot_word,100,input_length=max_seq_len-1))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(tot_word,activation='softmax'))

##compiling the model

model.compile(optimizer = "adam",loss = "categorical_crossentropy",metrics = ['accuracy'])
model.build(input_shape=(None, max_seq_len))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 14, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 14, 256)        │       365,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 128)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4818)           │       621,522 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,666,010 (6.36 MB)

 Trainable params: 1,666,010 (6.36 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history = model.fit(x_train,y_train,epochs = 20,validation_data=(x_test,y_test),verbose=1)

Epoch 1/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7138 - loss: 1.2691 - val_accuracy: 0.0505 - val_loss: 14.0189
Epoch 2/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7204 - loss: 1.2438 - val_accuracy: 0.0544 - val_loss: 14.0779
Epoch 3/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7200 - loss: 1.2336 - val_accuracy: 0.0548 - val_loss: 14.1652
Epoch 4/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7250 - loss: 1.2311 - val_accuracy: 0.0492 - val_loss: 14.2259
Epoch 5/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7283 - loss: 1.1961 - val_accuracy: 0.0534 - val_loss: 14.2961
Epoch 6/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7281 - loss: 1.1809 - val_accuracy: 0.0534 - val_loss: 14.3630
Epoch 7/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7376 - loss: 1.1558 - val_accuracy: 0.0511 - val_loss: 14.3900
Epoch 8/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7398 - loss: 1.1428 - val_accur

In [19]:
model.save("next_word_lstm.h5")

In [20]:
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None


In [28]:
inp_text = "i like your"

In [29]:
max_seq_len = model.input_shape[1]+1 ##maximum len of the input
next_word = predict_next_word(model,token,inp_text,max_seq_len)
next_word

'bride'